In [67]:
from tableaudocumentapi import Datasource
import numpy as np
import pandas as pd
import csv, os , datetime ,pprint
from tableaudocumentapi import Workbook
from tableaudocumentapi import *

### List of actions: 
    1. Work out a workbook list 
    2. Functionalise the workbook to datasources functions 
    3. Functionalise the datasources to DF 
    3. Collapse dfs into single df 

In [2]:
os.chdir('document-api-python-master\\samples\\replicate-workbook\\')
print(os.listdir())

['.ipynb_checkpoints', 'databases.csv', 'replicate_workbook.py', 'sample-superstore.twb', 'World Happiness.twb']


In [17]:
[print(n) for n in os.listdir() if n.endswith(('.twb'))]

sample-superstore.twb
World Happiness.twb


[None, None]

In [22]:
todo = [n for n  in os.listdir() if n.endswith(('.twb'))]
print(type(todo))
print(todo)

<class 'list'>
['sample-superstore.twb', 'World Happiness.twb']


In [40]:
def datasources2df(wbf,rds):
    ds1 = pd.DataFrame([f.__dict__.values() for f in rds.fields.values()], columns = [f.__dict__.keys() for f in rds.fields.values()][1])
    ds2 = pd.DataFrame([f for f in rds.fields.keys()],columns = ['name'])
    ds = ds2.join(ds1)
    #Clean Column Names:
    ds.columns = [col.replace('_','') for col in ds.columns]
    #Clean Variable Names: 
    ds['name'] = ds['name'].str.replace('[','').str.replace(']','')
    #Import workbook information:
    ds['datasoure_caption'] = str(rds.caption)
    ds['datasoure_connections'] = str(rds.connections)
    ds['datasoure_name'] = str(rds.name)
    ds['datasoure_version'] = str(rds.version)
    ds['from_wb'] = str(Workbook(wbf).filename)
    return ds

In [79]:
list4dfs = {}
for n, twb in enumerate(todo): 
    for ds in Workbook(twb).datasources:
        print ("Name : " ,Workbook(twb).filename)
        print (ds, " : " ,ds.name , '\n',ds.caption)
        print ([f.__dict__.keys() for f in ds.fields.values()][1])
        print ("ID : " ,n+1)
        list4dfs[n+1] = datasources2df(twb, ds)
        print (list4dfs[n+1].shape)
#ds

Name :  sample-superstore.twb
<tableaudocumentapi.datasource.Datasource object at 0x0000020BB6586208>  :  sqlserver.187oiqw1ls5tey1df9op71phda18 
 Orders (sample_superstore)
dict_keys(['_id', '_caption', '_datatype', '_role', '_type', '_alias', '_calculation', '_description', '_aggregation', '_worksheets'])
ID :  1
(25, 16)
Name :  World Happiness.twb
<tableaudocumentapi.datasource.Datasource object at 0x0000020BB6604A20>  :  federated.0ne8o6b1yp56zg1cblpoy03mpgbl 
 WorldHappniessCountries2015to2017
dict_keys(['_id', '_caption', '_datatype', '_role', '_type', '_alias', '_calculation', '_description', '_aggregation', '_worksheets'])
ID :  2
(40, 16)


In [80]:
totdf = pd.concat(list4dfs.values(), ignore_index=True)
print(totdf.shape)
pprint.pprint(totdf['datasoure_connections'].sample(5))

(65, 16)
28    ['<Connection server='' dbname='None' @ 0x20bb...
2     ['<Connection server='superstore.sample.db' db...
17    ['<Connection server='superstore.sample.db' db...
24    ['<Connection server='superstore.sample.db' db...
49    ['<Connection server='' dbname='None' @ 0x20bb...
Name: datasoure_connections, dtype: object


In [77]:
print(totdf['datasoure_connections'][3:45:6])

3     ['<Connection server='superstore.sample.db' db...
9     ['<Connection server='superstore.sample.db' db...
15    ['<Connection server='superstore.sample.db' db...
21    ['<Connection server='superstore.sample.db' db...
27    ['<Connection server='' dbname='None' @ 0x20bb...
33    ['<Connection server='' dbname='None' @ 0x20bb...
39    ['<Connection server='' dbname='None' @ 0x20bb...
Name: datasoure_connections, dtype: object


In [61]:
print(list4dfs.keys())
type(list4dfs)

dict_keys([1, 2])


dict